# Annnotated celery examples
This document shows both basic and more real-world examples of celery usage.

Relevant pages from celery's documentation
https://docs.celeryproject.org/en/stable/userguide/calling.html
https://docs.celeryproject.org/en/stable/userguide/canvas.html

## Asynchronous (i.e. remote and non-blocking) invocation of tasks

`orca.proj.boilerplate` is where the celery-ized tasks live. This section uses add(x, y) that adds two numbers to illustrate celery task invocation

In [1]:
import glob
from orca.proj import boilerplate

Start with a basic one, add numbers! `group` and `chain` are celery primitives that model a group of tasks or a chain of tasks (i.e. one depending on the previous one)

In [2]:
from celery import group, chain

In [3]:
type(boilerplate.add)

celery.local.PromiseProxy

In [4]:
# This runs on the current node
boilerplate.add(1, 1)

2

The quickiest way to run a task on a worker is by using the `delay()` method. Note that you can assign the `AsyncResult` to a variable and then use its `get()` method to get its results.

In [5]:
# So does this.
boilerplate.add.delay(1, 1)

<AsyncResult: aa9dc77f-28b9-4d64-bc27-608ea5d79e06>

## Embarassingly parallel operations: group

To add lots of pairs of numbers (an embarassingly parallel operation) you can call

In [6]:
group([boilerplate.add.s(a, a) for a  in [1,2,3,4,5,6,7]])()

<GroupResult: d74d4b59-02e8-46d5-82f2-ead29c5a8c66 [c0533143-36c1-496a-9aa5-099e0c9a59c2, 21e38a85-1ec3-457d-b35d-8bea205d8f5c, ddddbb38-ec05-432d-91f1-f65ba91ca5bc, 79271ff9-b1d3-4c81-955f-22db302cea4d, 17417b02-5366-4776-9b49-a13e7310214e, 2b220435-03c8-4123-8d71-b04a97289d16, 944d279d-08d5-493d-b13d-11f41325e7b6]>

Couple things happened there. `.s` is the method that creates the signatures for celery to execute, rather than executing the commands right away. `group()` creates the group of signatures (each can be done in parallel), and calling `()` on the `group` sends the group to the queue.

You can write a `for` loop and invoke `delay()` one by one to achieve the same thing as `group`, but `group` makes keeping track of the results easier.

In [7]:
# Sidenote: this runs on the current node
boilerplate.add.s(1, 1)()

2

In [8]:
# Sidenote2: this runs on the a worker and is equivalent to boilerplate.add.s.delay(1,1)
boilerplate.add.s(1, 1).delay()

<AsyncResult: bda5a418-31f6-429a-8a40-77f9c35b7f2e>

# Things that depend on each other: chain

The following executes one `add` at a time (not necessarily on the same worker!). Only the result of the last task gets returned


## Example 1

`si` stands for "immutable signature", which means "don't feed the previous call's results into me". The following returns 3

In [9]:
chain(boilerplate.add.si(0,1), boilerplate.add.si(1,1), boilerplate.add.si(1,2))().get()

3

## Example 2
Using `.s()` means that the previous task can feed its return value into the first argument of the next method.

The task str_concat is defined as follows
```
@app.task
def str_concat(first, second, third=''):
    return f'{first}{second}{third}'
```

In [10]:
chain(boilerplate.str_concat.s('a','b'), boilerplate.str_concat.s('c'))().get()

'abc'

In [11]:
chain(boilerplate.str_concat.s('a','b'), boilerplate.str_concat.s('c', 'd'))().get()

'abcd'

which is equivalent to

In [12]:
chain(boilerplate.str_concat.s('a','b'), boilerplate.str_concat.s('c', third='d'))().get()

'abcd'

*Warning* The following will get a
```
TypeError: str_concat() got multiple values for argument 'first'
```
on the task remotely.

In [13]:
chain(boilerplate.str_concat.s('a','b'), boilerplate.str_concat.s(first='d'))()

<AsyncResult: 70103357-9f3d-496b-b3a2-33b29851154e>

`chain` is not the most flexible thing ever as far as the syntax goes.

You can use the pipe operator to construct chains (much like bash)

In [14]:
(boilerplate.str_concat.s('a','b')| boilerplate.str_concat.s('c'))().get()

'abc'

# A recent example

The following code merges flags and correct scaling for sidereally separated pairs of measurement sets. And then integrate them.

In [3]:
DATA_DIR = '/lustre/yuping/0-100-hr-reduction/averaging-test/hh=03'

In [23]:
day1 = sorted(glob.glob(f'{DATA_DIR}/2018-03-22/2018*T*'))
day2 = sorted(glob.glob(f'{DATA_DIR}/2018-03-23/2018*T*'))

This is about 40 minutes of data

In [24]:
len(day1)

189

In [25]:
len(day2)

189

Verify that the same indexed elements in the two arrays are one sidereal day apart

In [10]:
day1[-1]

'/lustre/yuping/0-100-hr-reduction/averaging-test/hh=03/2018-03-22/2018-03-22T03:50:49'

In [11]:
day2[-1]

'/lustre/yuping/0-100-hr-reduction/averaging-test/hh=03/2018-03-23/2018-03-23T03:46:53'

The following queues a bunch of chains that merge the flags first and then do the gain scaling. I started a bunch of workers with concurrency 10 for this.

In [13]:
for i in range(len(day1)):
    for spw in range(22):
        ms1 = glob.glob(f'{day1[i]}/{spw:02d}_*ms')[0]
        ms2 = glob.glob(f'{day2[i]}/{spw:02d}_*ms')[0]
        (boilerplate.run_merge_flags.si(ms1, ms2) | boilerplate.run_correct_scaling.si(ms1, ms2))()

The following queues a bunch of concat runs. I changed the concurrency to 4 (see next section) before running this so that 4 concat runs on each node.

In [19]:
for spw in range(22):
    s = f'{spw:02d}'
    boilerplate.run_integrate_with_concat.delay(sorted(glob.glob(f'{DATA_DIR}/2018-03-22/2018*T*/{s}_*ms')),
                                            out_ms=f'{DATA_DIR}/2018-03-22/{s}_10min.ms')
    boilerplate.run_integrate_with_concat.delay(sorted(glob.glob(f'{DATA_DIR}/2018-03-23/2018*T*/{s}_*ms')),
                                            out_ms=f'{DATA_DIR}/2018-03-23/{s}_10min.ms')

# Worker control (still a work in progress)
Changing worker capacity, etc.

You should probably use the `start_workers.py` script to start workers across the cluster. The following commands show how you would change the concurrency for the workers (each node currently has one worker and each worker runs `concurrency` tasks in parallel via subprocessing.

In [14]:
from orca.proj.celery import app
from celery.app.control import Control
controller = Control(app)

You can increase the concurrency per node using `pool_grow(x)` where `x` is the number of concurrencies to add to each node

In [35]:
controller.pool_grow(7)

Shrinking capacity is still a work in progress. You can only use the following command when workers aren't doing work.

In [18]:
controller.pool_shrink(7)